<a href="https://colab.research.google.com/github/adampotton/MDM3-Rep-3/blob/main/inception_v3_aerial_trianing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting Google Drive and imports

In [2]:
from google.colab import files, drive
drive.mount('/content/drive')
import os
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading Numpy Data Arrays

In [3]:
#if you get an error the first time just run it again
os.chdir('../../../')
!pwd
os.chdir('content')
os.chdir(r'drive/MyDrive/Aerial Data')
print(os.listdir())

train_labels = np.load('train_labels.npy')
train_images = np.load('train_images.npy')
test_labels = np.load('test_labels.npy')
test_images = np.load('test_images.npy')
val_labels = np.load('val_labels.npy')
val_images = np.load('val_images.npy')

/
['aerial_pseudotsuga_menziesii', 'aerial_picea_abies', 'aerial_betula_spec', 'aerial_abies_alba', 'aerial_pinus_nigra', 'aerial_larix_decidua', 'aerial_larix_kaempferi', 'aerial_pinus_strobus', 'aerial_pinus_sylvestris', 'aerial_fraxinus_excelsior', 'aerial_fagus_sylvatica', 'aerial_acer_pseudoplatanus', 'aerial_quercus_robur', 'aerial_quercus_petraea', 'aerial_quercus_rubra', 'aerial_tilia_spec', 'aerial_populus_spec', 'aerial_prunus_spec', 'aerial_alnus_spec', 'train_images.npy', 'train_labels.npy', 'test_images.npy', 'test_labels.npy', 'val_images.npy', 'val_labels.npy']


# If Not using colab the following can be used to import the data

In [4]:
import gdown
import os


gdown.download(r'https://drive.google.com/file/d/1-7lEmQDmf06beJUD3NaQk82125NGqYpm/view?usp=drive_link', r'data/aerial_data/val_labels.npy',fuzzy=True)
gdown.download(r'https://drive.google.com/file/d/1-54Z1sN8uYgNG66MwWumRTlYMJmX7wWw/view?usp=drive_link', r'data/aerial_data/val_images.npy',fuzzy=True)
gdown.download(r'https://drive.google.com/file/d/1vKs-th9eHU_oMXTasVm-FD9Dximer82Q/view?usp=drive_link', r'data/aerial_data/train_labels.npy',fuzzy=True)
gdown.download(r'https://drive.google.com/file/d/1aJkI-7d5KCAAYCoXkrEcHVXvSsXc_-8n/view?usp=drive_link', r'data/aerial_data/train_images.npy',fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1-7lEmQDmf06beJUD3NaQk82125NGqYpm
To: c:\Users\bench\OneDrive\Documents\GitHub\MDM3-Rep-3\data\aerial_data\val_labels.npy
100%|██████████| 38.9k/38.9k [00:00<?, ?B/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-54Z1sN8uYgNG66MwWumRTlYMJmX7wWw
From (redirected): https://drive.google.com/uc?id=1-54Z1sN8uYgNG66MwWumRTlYMJmX7wWw&confirm=t&uuid=510fa68d-caf5-408a-9a59-45cf796a8dc1
To: c:\Users\bench\OneDrive\Documents\GitHub\MDM3-Rep-3\data\aerial_data\val_images.npy
100%|██████████| 171M/171M [00:14<00:00, 11.6MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1aJkI-7d5KCAAYCoXkrEcHVXvSsXc_-8n
From (redirected): https://drive.google.com/uc?id=1aJkI-7d5KCAAYCoXkrEcHVXvSsXc_-8n&confirm=t&uuid=337c523f-4708-43a3-8990-a3ffb1a8c052
To: c:\Users\bench\OneDrive\Documents\GitHub\MDM3-Rep-3\data\aerial_data\train_labels.npy
100%|██████████| 1.36G/1.36G [01:57<00:00, 11.6MB/s]
Downloading...
From: https://dri

In [18]:
import os
import numpy as np
folder = 'data/aerial_data'
train_images = np.load(os.path.join(folder, 'train_images.npy'))
train_labels = np.load(os.path.join(folder, 'train_labels.npy'))
val_images = np.load(os.path.join(folder, 'val_images.npy'))
val_labels = np.load(os.path.join(folder, 'val_labels.npy'))

In [19]:
#inspecting data
print("train data:")
print("images shape:")
print(train_images.shape)
print("labels shape:")
print(train_labels.shape)

#print("test data:")
#print("images shape:")
#print(test_images.shape)
#print("labels shape:")
#print(test_labels.shape)

print("val data:")
print("images shape:")
print(val_images.shape)
print("labels shape:")
print(val_labels.shape)


train data:
images shape:
(3688, 304, 304, 4)
labels shape:
(3688,)
val data:
images shape:
(462, 304, 304, 4)
labels shape:
(462,)


# Performing Feature Extraction on Inception-V3, which is currently trained on image net
Adapted from: <https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/finetuning_torchvision_models_tutorial.ipynb#scrollTo=NiFT4EYmprqG>

# Training Parameters

In [11]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

# Number of tree species categories
num_classes = 19

# Batch size for training (should be a power of 2 if training on the GPU and is limited by memory)
batch_size = 4

# Number of epochs to train for
num_epochs = 15

#True if we are just training the output layer of the network
feature_extract = True

print(torch.cuda.is_available())

C:\Users\bench\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False


# Function For training the model

In [13]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=True):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            #deep copying the model if its performance has improved
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print("epoch {} complete".format(epoch))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights and return best model
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

# Ensuring that model parameters are not used during training since we are only interested in the final layer

In [14]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# Defining function to reshape the model to account for the number of classes
By default Inception-v3 has 1000 classes as it is trained on imageNet, whilst our dataset has 19

In [15]:
def initialize_inception_model(num_classes, feature_extract, use_pretrained=True, model=None):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    """ Inception v3
    Be careful, expects (299,299) sized images and has auxiliary output
    """
    if model is None:
      model_ft = models.inception_v3(pretrained=use_pretrained)
    else:
      model_ft = model
    #"switching off" training on the majority of the model weights if feature_extract
    set_parameter_requires_grad(model_ft, feature_extract)
    # Handle the auxilary net
    num_ftrs = model_ft.AuxLogits.fc.in_features
    model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
    # Handle the primary net
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs,num_classes)
    input_size = 299

    return model_ft, input_size


# Initializing Model


In [16]:
#initializing model
#Initialize the model for this run
model_ft, input_size = initialize_inception_model(num_classes, feature_extract, use_pretrained=True)

C:\Users\bench\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\bench\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\bench/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth
100%|██

# Formatting Loaded Data and creating data loader

In [20]:
# normalization for training and validation
# hardcoded normalization values for the model
# right now validation and training transform is the same and no augmentation
image_transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

print("Initializing Datasets and Dataloaders...")

#creating a custom pytorch dataset for the numpy arrays we loaded earlier
#with applied transforms
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = torch.from_numpy(images)
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx,:,:,:]
        label = self.labels[idx]

        if self.transform:
          image = self.transform(image)

        return image, label


#creating training and validation pytorch datasets
training_dataset = CustomDataset(train_images[:,:,:,1:], train_labels, image_transform)
val_dataset = CustomDataset(val_images[:,:,:,1:], val_labels, image_transform)

# Create training and validation dataloaders
dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True),
                    'val': torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)}


Initializing Datasets and Dataloaders...


# Training the model

In [21]:
## Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("sending model to",device)
# Send the model to GPU
model_ft = model_ft.to(device)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

#Creating the Optimizer
if feature_extract:
  params_to_update = []
  for name,param in model_ft.named_parameters():
    if param.requires_grad==True:
      params_to_update.append(param)
      print("\t",name)
else:
  params_to_update = model_ft.parameters()

print("number of parameters to train =",len(params_to_update))
print("params to train =")
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)





sending model to cpu
	 AuxLogits.fc.weight
	 AuxLogits.fc.bias
	 fc.weight
	 fc.bias
number of parameters to train = 4
params to train =


In [22]:
# Train and evaluate
model_ft, hist = train_model(model_ft,
                             dataloaders_dict,
                             criterion,
                             optimizer_ft,
                             num_epochs=num_epochs,
                             is_inception=True)

torch.save(model_ft.state_dict(),'trained_model.pth')

Epoch 0/14
----------


TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

#Testing Eval on the inception model

In [6]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.eval()
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")

try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
  output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


tensor([ 1.5696e-01,  1.1051e-01, -4.4363e-01, -6.1476e-02, -2.0093e-01,
         1.6820e-01,  5.8117e-01,  6.0333e-02, -7.7650e-02, -1.1192e+00,
        -2.1800e-01, -5.2624e-01, -1.1816e-01,  6.6276e-02,  8.4988e-01,
        -5.3260e-02, -4.7113e-01, -2.5986e-03,  2.7504e-01,  1.7060e-01,
         4.4231e-01, -3.8615e-01,  1.3482e-01, -5.2201e-01, -3.3595e-04,
        -2.3492e-01, -2.3264e-01, -7.1890e-03,  1.6899e-02, -1.2724e-01,
         2.0508e-01, -6.9896e-03,  1.3074e+00, -2.3181e-01,  9.8966e-01,
        -7.7783e-01,  4.8900e-01, -4.2244e-01, -2.6523e-01, -1.5005e-01,
         3.5147e-01, -1.7462e-01,  1.0251e-01,  4.2223e-01, -1.4342e-01,
         1.3578e-02, -4.6477e-01,  3.5966e-01,  1.5194e-01,  2.0217e-01,
        -1.3763e-01,  1.4986e-01,  2.8415e-01, -3.7086e-01,  9.7688e-01,
         7.9543e-01, -7.4032e-01, -7.8853e-02, -9.6364e-02,  1.2193e-01,
        -1.2712e-01,  3.6825e-01, -4.5354e-01, -7.4801e-01, -7.9353e-02,
         2.1689e-01,  2.7018e-01,  1.0089e+00,  4.0